In [1]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, FloatSlider, IntSlider, Dropdown, VBox, HBox, Button, Output, FileUpload
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import functions from run.py
from run import symbol_detection_pipeline, visualize_results


In [2]:
# Load templates and reference colors
pickle_path = "data/templates.pkl"

with open(pickle_path, "rb") as f:
    reference_colors, all_templates, color_template_freq = pickle.load(f)
print(f"Loaded {len(all_templates)} templates and {len(reference_colors)} reference colors")



Loaded 24 templates and 4 reference colors


In [3]:
# Create parameter widgets
image_path_widget = widgets.Text(
    value="data/images/positive_00000.jpeg",
    description="Image Path:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

ncc_threshold_widget = FloatSlider(
    value=0.9,
    min=0.0,
    max=1.0,
    step=0.01,
    description='NCC Threshold:',
    style={'description_width': 'initial'}
)

rgb_threshold_widget = FloatSlider(
    value=0.3,
    min=0.0,
    max=1.0,
    step=0.01,
    description='RGB Threshold:',
    style={'description_width': 'initial'}
)

black_val_threshold_widget = FloatSlider(
    value=50,
    min=0,
    max=255,
    step=1,
    description='Black Val Threshold:',
    style={'description_width': 'initial'}
)

white_sat_threshold_widget = FloatSlider(
    value=0.1,
    min=0.0,
    max=1.0,
    step=0.01,
    description='White Sat Threshold:',
    style={'description_width': 'initial'}
)

white_val_threshold_widget = FloatSlider(
    value=100,
    min=0,
    max=255,
    step=1,
    description='White Val Threshold:',
    style={'description_width': 'initial'}
)

min_line_length_percent_widget = FloatSlider(
    value=0.03,
    min=0.01,
    max=0.2,
    step=0.01,
    description='Min Line Length %:',
    style={'description_width': 'initial'}
)

adjacency_radius_widget = FloatSlider(
    value=6,
    min=1,
    max=50,
    step=1,
    description='Adjacency Radius:',
    style={'description_width': 'initial'}
)

dedup_thresh_widget = FloatSlider(
    value=100,
    min=1,
    max=200,
    step=1,
    description='Dedup Threshold:',
    style={'description_width': 'initial'}
)

scale_widget = IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='LSD Scale:',
    style={'description_width': 'initial'}
)

num_octaves_widget = IntSlider(
    value=4,
    min=1,
    max=8,
    step=1,
    description='Num Octaves:',
    style={'description_width': 'initial'}
)

angle_tolerance_widget = FloatSlider(
    value=5,
    min=1,
    max=30,
    step=1,
    description='Angle Tolerance:',
    style={'description_width': 'initial'}
)

distance_tolerance_widget = FloatSlider(
    value=2,
    min=1,
    max=50,
    step=1,
    description='Distance Tolerance:',
    style={'description_width': 'initial'}
)


In [4]:
# Create output widget for results
output = Output()

def run_pipeline_and_visualize():
    """Run the pipeline with current parameter values and visualize results"""
    with output:
        clear_output(wait=True)
        
        # Check if image path exists
        if not os.path.exists(image_path_widget.value):
            print(f"Error: Image path '{image_path_widget.value}' does not exist.")
            return
        
        if not all_templates:
            print("Error: No templates loaded. Please check the pickle file path.")
            return
        
        try:
            print("Running pipeline with current parameters...")
            
            # Run pipeline
            results = symbol_detection_pipeline(
                image_path_widget.value,
                all_templates,
                reference_colors,
                rgb_threshold=rgb_threshold_widget.value,
                black_val_threshold=black_val_threshold_widget.value,
                white_sat_threshold=white_sat_threshold_widget.value,
                white_val_threshold=white_val_threshold_widget.value,
                ncc_threshold=ncc_threshold_widget.value,
                min_line_length_percent=min_line_length_percent_widget.value,
                adjacency_radius=adjacency_radius_widget.value,
                dedup_thresh=dedup_thresh_widget.value,
                scale=scale_widget.value,
                num_octaves=num_octaves_widget.value,
                angle_tolerance=angle_tolerance_widget.value,
                distance_tolerance=distance_tolerance_widget.value,
                color_template_freq=color_template_freq
            )
            
            # Visualize results
            fig = visualize_results(image_path_widget.value, results)
            for i, r in enumerate(results['matches']):
              print(f"- Match {i} - {r['template_info']['name']} - {r['ncc_score']}")
            plt.show()
            
            # Print summary
            print(f"\nPipeline Results:")
            print(f"- Found {len(results['matches'])} matches")
            print(f"- Processed {results['num_candidates']} candidate quadrilaterals")
            print(f"- Checked against {results['num_templates']} templates")
            print(f"- Total template attempts: {results['template_attempts']}")
            
        except Exception as e:
            print(f"Error running pipeline: {str(e)}")
            import traceback
            traceback.print_exc()

# Create run button
run_button = Button(
    description='Run Pipeline',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)
run_button.on_click(lambda b: run_pipeline_and_visualize())


In [5]:
# Organize widgets into groups
color_widgets = VBox([
    widgets.HTML("<h3>Color Splitting Parameters</h3>"),
    rgb_threshold_widget,
    black_val_threshold_widget,
    white_sat_threshold_widget,
    white_val_threshold_widget
])

line_widgets = VBox([
    widgets.HTML("<h3>Line Detection Parameters</h3>"),
    min_line_length_percent_widget,
    scale_widget,
    num_octaves_widget,
    angle_tolerance_widget,
    distance_tolerance_widget
])

matching_widgets = VBox([
    widgets.HTML("<h3>Matching Parameters</h3>"),
    ncc_threshold_widget,
    adjacency_radius_widget,
    dedup_thresh_widget
])

# Create the main interface
interface = VBox([
    widgets.HTML("<h2>Interactive Symbol Detection Pipeline</h2>"),
    image_path_widget,
    run_button,
    HBox([color_widgets, line_widgets, matching_widgets]),
    output
])

# Display the interface
display(interface)


In [6]:
# Optional: Create preset configurations
presets = {
    'Default': {
        'ncc_threshold': 0.9,
        'rgb_threshold': 0.1,
        'black_val_threshold': 50,
        'white_sat_threshold': 0.2,
        'white_val_threshold': 180,
        'min_line_length_percent': 0.05,
        'adjacency_radius': 5,
        'dedup_thresh': 100,
        'scale': 2,
        'num_octaves': 4,
        'angle_tolerance': 5,
        'distance_tolerance': 5
    },
    'High Sensitivity': {
        'ncc_threshold': 0.7,
        'rgb_threshold': 0.15,
        'black_val_threshold': 60,
        'white_sat_threshold': 0.25,
        'white_val_threshold': 170,
        'min_line_length_percent': 0.03,
        'adjacency_radius': 8,
        'dedup_thresh': 80,
        'scale': 1.5,
        'num_octaves': 5,
        'angle_tolerance': 8,
        'distance_tolerance': 8
    },
    'High Precision': {
        'ncc_threshold': 0.95,
        'rgb_threshold': 0.05,
        'black_val_threshold': 40,
        'white_sat_threshold': 0.15,
        'white_val_threshold': 190,
        'min_line_length_percent': 0.08,
        'adjacency_radius': 3,
        'dedup_thresh': 50,
        'scale': 2.5,
        'num_octaves': 3,
        'angle_tolerance': 3,
        'distance_tolerance': 3
    }
}

def load_preset(preset_name):
    """Load a preset configuration"""
    if preset_name in presets:
        config = presets[preset_name]
        ncc_threshold_widget.value = config['ncc_threshold']
        rgb_threshold_widget.value = config['rgb_threshold']
        black_val_threshold_widget.value = config['black_val_threshold']
        white_sat_threshold_widget.value = config['white_sat_threshold']
        white_val_threshold_widget.value = config['white_val_threshold']
        min_line_length_percent_widget.value = config['min_line_length_percent']
        adjacency_radius_widget.value = config['adjacency_radius']
        dedup_thresh_widget.value = config['dedup_thresh']
        scale_widget.value = config['scale']
        num_octaves_widget.value = config['num_octaves']
        angle_tolerance_widget.value = config['angle_tolerance']
        distance_tolerance_widget.value = config['distance_tolerance']
        print(f"Loaded preset: {preset_name}")

preset_dropdown = Dropdown(
    options=list(presets.keys()),
    description='Load Preset:',
    style={'description_width': 'initial'}
)

load_preset_button = Button(
    description='Load',
    button_style='info',
    layout=widgets.Layout(width='100px')
)
load_preset_button.on_click(lambda b: load_preset(preset_dropdown.value))

preset_interface = HBox([
    preset_dropdown,
    load_preset_button
])

print("Preset configurations:")
display(preset_interface)


Preset configurations:
